In [1]:
%pip install tqdm langchain langchain-core langchain-community langchain-text-splitters langchain-openai langchain-pinecone docx2txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
from dotenv import load_dotenv

load_dotenv()

True

### 청킹 작업 

In [3]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

loader = Docx2txtLoader("./tax.docx")
document_list = loader.load_and_split(text_splitter=text_splitter)
len(document_list)

225

In [4]:
document_list[52]

Document(metadata={'source': './tax.docx'}, page_content='제55조(세율) ①거주자의 종합소득에 대한 소득세는 해당 연도의 종합소득과세표준에 다음의 세율을 적용하여 계산한 금액(이하 “종합소득산출세액”이라 한다)을 그 세액으로 한다. <개정 2014. 1. 1., 2016. 12. 20., 2017. 12. 19., 2020. 12. 29., 2022. 12. 31.>\n\n| 종합소득 과세표준          | 세율                                         |\n\n|-------------------|--------------------------------------------|\n\n| 1,400만원 이하     | 과세표준의 6퍼센트                             |\n\n| 1,400만원 초과     5,000만원 이하     | 84만원 + (1,400만원을 초과하는 금액의 15퍼센트)  |\n\n| 5,000만원 초과   8,800만원 이하     | 624만원 + (5,000만원을 초과하는 금액의 24퍼센트) |\n\n| 8,800만원 초과 1억5천만원 이하    | 3,706만원 + (8,800만원을 초과하는 금액의 35퍼센트)|\n\n| 1억5천만원 초과 3억원 이하         | 3,706만원 + (1억5천만원을 초과하는 금액의 38퍼센트)|\n\n| 3억원 초과    5억원 이하         | 9,406만원 + (3억원을 초과하는 금액의 38퍼센트)   |\n\n| 5억원 초과      10억원 이하        | 1억 7,406만원 + (5억원을 초과하는 금액의 42퍼센트)|\n\n| 10억원 초과        | 3억 8,406만원 + (10억원을 초과하는 금액의 45퍼센트)|\n\n\n\n\n\n② 거주자의 퇴직소득에 대한 소득세는 다음 각 호의 순서에 따라 계산한 금액(이하 “퇴직소득 산출세액”이라 한다)으로 한다.<개정 20

### 임베딩 작업 

In [5]:
from langchain_openai import OpenAIEmbeddings

embedding = OpenAIEmbeddings(
    model='text-embedding-3-large'
) # 3072차원

### vector DB

#### 이미 생성한 후 로딩용

In [6]:
# import os
# from pinecone import Pinecone
# from langchain_pinecone import PineconeVectorStore

# index_name = 'tax-index'
# pinecone_api_key = os.getenv('PINECONE_API_KEY')
# pc = Pinecone(api_key=pinecone_api_key)

# vectorstor=PineconeVectorStore.from_existing_index(
#     index_name=index_name,
#     embedding=embedding
# )

#### 처음 생성용

In [7]:
import os
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

index_name = 'tax-index'
pinecone_api_key = os.getenv('PINECONE_API_KEY')
pc = Pinecone(api_key=pinecone_api_key)
pinecone_index = pc.Index(index_name)

vectorstore = PineconeVectorStore(
    index=pinecone_index,
    embedding=embedding,
    text_key='text'
)

texts = [doc.page_content for doc in document_list]
metadatas = [doc.metadata for doc in document_list]

batch_size = 2

for i in range(0, len(texts), batch_size):
    batch_texts = texts[i:i + batch_size]
    batch_metadatas = metadatas[i:i + batch_size]
    
    try:
        vectorstore.add_texts(texts=batch_texts, metadatas=batch_metadatas)
    except Exception as e:
        print(f"Error adding batch {i // batch_size + 1}: {e}")

c:\AI_prompt\second\AI_Agent\ai_agent_work2\tax_env\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
query = '연봉 5천만원인 거주자의 종합소득세는?'

In [9]:
retriever = vectorstore.as_retriever(search_kwargs={'k':4})
retriever.invoke(query)

[Document(id='65163488-03ba-4f0f-8536-729598217594', metadata={'source': './tax.docx'}, page_content='제55조(세율) ①거주자의 종합소득에 대한 소득세는 해당 연도의 종합소득과세표준에 다음의 세율을 적용하여 계산한 금액(이하 “종합소득산출세액”이라 한다)을 그 세액으로 한다. <개정 2014. 1. 1., 2016. 12. 20., 2017. 12. 19., 2020. 12. 29., 2022. 12. 31.>\n\n| 종합소득 과세표준          | 세율                                         |\n\n|-------------------|--------------------------------------------|\n\n| 1,400만원 이하     | 과세표준의 6퍼센트                             |\n\n| 1,400만원 초과     5,000만원 이하     | 84만원 + (1,400만원을 초과하는 금액의 15퍼센트)  |\n\n| 5,000만원 초과   8,800만원 이하     | 624만원 + (5,000만원을 초과하는 금액의 24퍼센트) |\n\n| 8,800만원 초과 1억5천만원 이하    | 3,706만원 + (8,800만원을 초과하는 금액의 35퍼센트)|\n\n| 1억5천만원 초과 3억원 이하         | 3,706만원 + (1억5천만원을 초과하는 금액의 38퍼센트)|\n\n| 3억원 초과    5억원 이하         | 9,406만원 + (3억원을 초과하는 금액의 38퍼센트)   |\n\n| 5억원 초과      10억원 이하        | 1억 7,406만원 + (5억원을 초과하는 금액의 42퍼센트)|\n\n| 10억원 초과        | 3억 8,406만원 + (10억원을 초과하는 금액의 45퍼센트)|\n\n\n\n\n\n② 거주자의 퇴직소득에 대한 소득세는 다음 각 호의 순서

In [10]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

c:\AI_prompt\second\AI_Agent\ai_agent_work2\tax_env\myenv\Lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [11]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [12]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model='gpt-4o-mini'
)

### Retriever

In [13]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type_kwargs={"prompt":prompt},
)

In [14]:
query = '연봉 5천만원인 직장인의 종합소득세는?'

In [15]:
ai_message = qa_chain.invoke({"query":query})

In [16]:
ai_message

{'query': '연봉 5천만원인 직장인의 종합소득세는?',
 'result': '연봉 5천만원인 직장인의 종합소득세는 근로소득에 대한 기본세율이 적용됩니다. 기본세율은 구체적 소득 상황에 따라 달라지지만, 일반적으로 국민세와 지방세를 포함하여 일정 비율로 계산됩니다. 따라서 정확한 세액은 추가적인 세액 공제 및 개인 사정에 따라 달라질 수 있습니다.'}

### prompt

In [17]:
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate

dictionary = ["사람을 나타내는 표현 -> 거주자"]

prompt = ChatPromptTemplate.from_template(f"""
사용자의 질문을 보고, 우리의 사전을 참고해서 사용자의 질문을 변경해 주세요.
만약 변경할 필요가 없다고 판단된다면, 사용자의 질문을 변경하지 않아도 됩니다.
그런 경우에는 질문만 리턴해 주세요.
사전: {dictionary}
질문: {{question}}
""")

dictionary_chain = prompt | llm | StrOutputParser()

tax_chain = {"query": dictionary_chain} | qa_chain

In [18]:
new_question = dictionary_chain.invoke({"question": query})
new_question

'연봉 5천만원인 거주자의 종합소득세는?'

In [19]:
ai_message = tax_chain.invoke({"question":query})
ai_message

{'query': '연봉 5천만원인 거주자의 종합소득세는?',
 'result': '연봉 5천만원인 거주자의 종합소득세는 84만원에 1,400만원을 초과하는 금액의 15%를 추가한 금액으로 계산됩니다. 이를 통해 총 세액은 84만원 + (5천만원 - 1,400만원) × 0.15가 됩니다. 정확한 금액은 계산을 통해 확인할 수 있습니다.'}

### Streamlit